<a href="https://colab.research.google.com/github/jarl24-dev/stock-markets-analytics-zoomcamp/blob/main/01-intro-and-data-sources/Homework1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1: [Index] S&P 500 Stocks Added to the Index

In [ ]:
import pandas as pd

In [ ]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
print(f"{len(tables)} tablas encontradas")
df = tables[0]
df.head()

2 tablas encontradas


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [ ]:
df.dtypes

,0
Symbol,object
Security,object
GICS Sector,object
GICS Sub-Industry,object
Headquarters Location,object
Date added,object
CIK,int64
Founded,object


In [ ]:
df['Date added'] = pd.to_datetime(df['Date added'])
df.dtypes

,0
Symbol,object
Security,object
GICS Sector,object
GICS Sub-Industry,object
Headquarters Location,object
Date added,datetime64[ns]
CIK,int64
Founded,object


In [ ]:
df['year'] = df['Date added'].dt.year
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,year
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,1957
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,2017
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,1957
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),2012
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,2011


In [ ]:
df['year'].value_counts().sort_values(ascending=False).head(5)

,count
year,
1957,53
2017,23
2016,23
2019,22
2008,17


In [ ]:
df.groupby('year').size().reset_index(name='Stocks_Added_per_Year').sort_values(by='Stocks_Added_per_Year', ascending=False).head(5)

,year,Stocks_Added_per_Year
0,1957,53
47,2016,23
48,2017,23
50,2019,22
39,2008,17


# Question 2. [Macro] Indexes YTD (as of 1 May 2025)

In [ ]:
!pip install yfinance
!pip install pandas_datareader


In [ ]:
import yfinance as yf

# Índices y sus símbolos
indices = {
    "United States - S&P 500": "^GSPC",
  "China - Shanghai Composite": "000001.SS",
  "Hong Kong - HANG SENG INDEX": "^HSI",
  "Australia - S&P/ASX 200": "^AXJO",
  "India - Nifty 50": "^NSEI",
  "Canada - S&P/TSX Composite": "^GSPTSE",
  "Germany - DAX": "^GDAXI",
  "United Kingdom - FTSE 100": "^FTSE",
  "Japan - Nikkei 225": "^N225",
  "Mexico - IPC Mexico": "^MXX",
  "Brazil - Ibovespa": "^BVSP"
}

start_date = "2025-01-01"
end_date = "2025-05-01"

df_result = pd.DataFrame(columns=["Nombre", "YTD"])

for name, ticker in indices.items():
  ticker_obj = yf.Ticker(ticker)
  df = ticker_obj.history(start = start_date, end = end_date )

  if not df.empty:

    start_price = df['Close'].iloc[0]
    end_price = df['Close'].iloc[-1]

    ytd = ((end_price - start_price) / start_price) * 100
    #print(f"{name}: {ytd}%")

    nuevas_filas = pd.DataFrame([{"Nombre": name, "YTD": ytd}])
    df_result = pd.concat([df_result, nuevas_filas], ignore_index=True)

  else:
    print(f"{name}: No se pudo obtener datos.")


<ipython-input-91-99442b604f8b>:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_result = pd.concat([df_result, nuevas_filas], ignore_index=True)


In [ ]:
df_result.sort_values(by='YTD', ascending=False)

,Nombre,YTD
9,Mexico - IPC Mexico,13.049444
2,Hong Kong - HANG SENG INDEX,12.720018
10,Brazil - Ibovespa,12.438710
6,Germany - DAX,12.346378
7,United Kingdom - FTSE 100,2.842590
4,India - Nifty 50,2.490424
1,China - Shanghai Composite,0.504817
5,Canada - S&P/TSX Composite,-0.226126
3,Australia - S&P/ASX 200,-0.914500
0,United States - S&P 500,-5.103301


# Question 3. [Index] S&P 500 Market Corrections Analysis

In [ ]:

import yfinance as yf

# Download S&P 500 historical data (1950-present) using yfinance
sp500 = yf.Ticker("^GSPC")
hist = sp500.history(start="1950-01-01")
hist['All-Time High'] = hist['Close'].cummax()
hist['IsAllTimeHigh']=hist['Close'] == hist['All-Time High']

start='2007-10-09'
end='2009-03-09'
hist.loc[start:end]

,Open,High,Low,Close,Volume,Dividends,Stock Splits,All-Time High,IsAllTimeHigh
Date,,,,,,,,,
2007-10-09 00:00:00-04:00,1553.180054,1565.260010,1551.819946,1565.150024,2932040000,0.0,0.0,1565.150024,True
2007-10-10 00:00:00-04:00,1564.979980,1565.420044,1555.459961,1562.469971,3044760000,0.0,0.0,1565.150024,False
2007-10-11 00:00:00-04:00,1564.719971,1576.089966,1546.719971,1554.410034,3911260000,0.0,0.0,1565.150024,False
2007-10-12 00:00:00-04:00,1555.410034,1563.030029,1554.089966,1561.800049,2788690000,0.0,0.0,1565.150024,False
2007-10-15 00:00:00-04:00,1562.250000,1564.739990,1540.810059,1548.709961,3139290000,0.0,0.0,1565.150024,False
...,...,...,...,...,...,...,...,...,...
2009-03-03 00:00:00-05:00,704.440002,711.669983,692.299988,696.330017,7583230000,0.0,0.0,1565.150024,False
2009-03-04 00:00:00-05:00,698.599976,724.119995,698.599976,712.869995,7673620000,0.0,0.0,1565.150024,False
2009-03-05 00:00:00-05:00,708.270020,708.270020,677.929993,682.549988,7507250000,0.0,0.0,1565.150024,False


In [ ]:
all_time_highs = hist[hist['Close'] == hist['All-Time High']]

# Encontrar mínimos entre cada par de máximos históricos consecutivos
drawdown_percentages = []
for i in range(len(all_time_highs) - 1):
  start = all_time_highs.index[i]
  end = all_time_highs.index[i + 1]

  min_price = hist.loc[start:end, 'Close'].min()
  max_price = hist.loc[start, 'Close']

  drawdown_percentage= ((max_price- min_price) / max_price) * 100

  drawdown_percentages.append({
      'Max Date': start,
      'Min Date': hist.loc[start:end, 'Close'].idxmin(),
      'Drawdown(%)': drawdown_percentage,
      'duration': hist.loc[start:end, 'Close'].idxmin() - start
      })
df_result = pd.DataFrame(drawdown_percentages)
df_result = df_result[df_result['Drawdown(%)'] > 5]

In [ ]:
df_result.sort_values(by='Drawdown(%)', ascending=False)

#2007-10-09 to 2009-03-09: 56.8% drawdown over 517 days
#2000-03-24 to 2002-10-09: 49.1% drawdown over 929 days
#1973-01-11 to 1974-10-03: 48.2% drawdown over 630 days
#1968-11-29 to 1970-05-26: 36.1% drawdown over 543 days
#2020-02-19 to 2020-03-23: 33.9% drawdown over 33 days
#1987-08-25 to 1987-12-04: 33.5% drawdown over 101 days
#1961-12-12 to 1962-06-26: 28.0% drawdown over 196 days
#1980-11-28 to 1982-08-12: 27.1% drawdown over 622 days
#2022-01-03 to 2022-10-12: 25.4% drawdown over 282 days
#1966-02-09 to 1966-10-07: 22.2% drawdown over 240 days

,Max Date,Min Date,Drawdown(%),duration
1067,2007-10-09 00:00:00-04:00,2009-03-09 00:00:00-04:00,56.775388,517 days 00:00:00
1058,2000-03-24 00:00:00-05:00,2002-10-09 00:00:00-04:00,49.146948,928 days 23:00:00
554,1973-01-11 00:00:00-05:00,1974-10-03 00:00:00-04:00,48.203593,629 days 23:00:00
519,1968-11-29 00:00:00-05:00,1970-05-26 00:00:00-04:00,36.061641,542 days 23:00:00
1322,2020-02-19 00:00:00-05:00,2020-03-23 00:00:00-04:00,33.924960,32 days 23:00:00
...,...,...,...,...
568,1980-09-22 00:00:00-04:00,1980-09-29 00:00:00-04:00,5.260731,7 days 00:00:00
1396,2021-09-02 00:00:00-04:00,2021-10-04 00:00:00-04:00,5.212538,32 days 00:00:00
535,1972-08-14 00:00:00-04:00,1972-10-16 00:00:00-04:00,5.135501,63 days 00:00:00
1033,1999-04-12 00:00:00-04:00,1999-04-19 00:00:00-04:00,5.089688,7 days 00:00:00


In [ ]:
df_result['duration'].describe()

,duration
count,71
mean,113 days 02:17:44.788732394
std,179 days 01:33:15.692337412
min,7 days 00:00:00
25%,21 days 12:00:00
50%,39 days 00:00:00
75%,89 days 00:00:00
max,928 days 23:00:00


# Question 4. [Stocks] Earnings Surprise Analysis for Amazon (AMZN)

In [101]:
import pandas as pd
from dateutil import parser

url = "https://raw.githubusercontent.com/DataTalksClub/stock-markets-analytics-zoomcamp/main/cohorts/2025/ha1_Amazon.csv"
df = pd.read_csv(url, delimiter=';')

cols_to_convert = ['EPS Estimate', 'Reported EPS', 'Surprise (%)']
for col in cols_to_convert:
    df[col] = pd.to_numeric(df[col].str.replace('[^-.0-9]', '', regex=True), errors='coerce')

df["Earnings Date"] = df["Earnings Date"].apply(lambda x: parser.parse(x) if pd.notna(x) else pd.NaT)
df["Date"] = df["Earnings Date"].dt.date

df = df.drop("Earnings Date", axis=1)

df.head()

/usr/local/lib/python3.11/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/local/lib/python3.11/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,Symbol,Company,EPS Estimate,Reported EPS,Surprise (%),Date
0,AMZN,Amazon.com Inc,NaN,NaN,NaN,2026-04-29
1,AMZN,Amazon.com Inc,NaN,NaN,NaN,2026-02-04
2,AMZN,Amazon.com Inc,NaN,NaN,NaN,2025-10-29
3,AMZN,Amazon.com Inc,NaN,NaN,NaN,2025-07-30
4,AMZN,"Amazon.com, Inc.",0.36,0.59,16.74,2025-05-01


In [102]:
import yfinance as yf

amzn = yf.Ticker("AMZN")

hist = amzn.history(period='max')

hist['2-Day percentage changes'] = (hist['Close'].shift(-1) / hist['Close'].shift(1)) - 1
hist.index = hist.index.date

hist.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,2-Day percentage changes
1997-05-15,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0,NaN
1997-05-16,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0,-0.127659
1997-05-19,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0,-0.054211
1997-05-20,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0,-0.164639
1997-05-21,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0,-0.146494


In [103]:
df_positive_surprise = df[df["Reported EPS"] > df["EPS Estimate"]]
df_positive_surprise.set_index('Date', inplace=True)
df_positive_surprise.head()

,Symbol,Company,EPS Estimate,Reported EPS,Surprise (%)
Date,,,,,
2025-05-01,AMZN,"Amazon.com, Inc.",0.36,0.59,16.74
2025-02-06,AMZN,"Amazon.com, Inc.",0.49,0.86,24.47
2024-10-31,AMZN,"Amazon.com, Inc.",0.14,0.43,25.17
2024-04-30,AMZN,"Amazon.com, Inc.",0.83,0.98,17.91
2024-02-01,AMZN,"Amazon.com, Inc.",0.80,1.00,24.55


In [108]:
df_merged = hist.join(df_positive_surprise, how='inner')
print("Median 2-day return after positive surprises:",(df_merged['2-Day percentage changes']*100).median())
print("Median 2-day return for all trading days:",(hist['2-Day percentage changes']*100).median())

Median 2-day return after positive surprises: 2.6341689747185426
Median 2-day return for all trading days: 0.16538153051188775
